# Step 07: Execute Portfolio Mapping Batch

This notebook executes SQL scripts to create sub-portfolios in Moody's Risk Modeler.

**Tasks:**
- Retrieve Portfolio Mapping batch from Stage 01
- Review batch configuration and job details
- Execute SQL mapping scripts for each portfolio
- Display results and update job status

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch, validate_batch
from helpers.database import execute_query
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType

# Flag to track validation state - allows notebook to complete gracefully on failure
validation_failed = False
validation_errors = []

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_07_Execute_Portfolio_Mapping (chained).ipynb')

# Display context
ux.header("Portfolio Mapping Execution")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"✓ Step tracking initialized for '{context.step_name}'")

## 2) Retrieve Portfolio Mapping Batch

In [ ]:
# Retrieve Portfolio Mapping batch from Stage_01/Step_03
ux.subheader("Retrieve Portfolio Mapping Batch")

# Query for Stage_01/Step_03 step run
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

if BatchType.PORTFOLIO_MAPPING not in batches:
    raise ValueError(f"Portfolio Mapping batch not found. Available: {list(batches.keys())}")

mapping_batch_id = int(batches[BatchType.PORTFOLIO_MAPPING])

ux.success(f"✓ Retrieved Portfolio Mapping batch: ID={mapping_batch_id}")
step.log(f"Retrieved Portfolio Mapping batch: ID={mapping_batch_id}")

## 3) Review Batch Configuration

In [ ]:
# Verify batch status and display job information
ux.subheader("Verify Batch Status")

# Read batch details
batch = read_batch(mapping_batch_id)

batch_info = [
    ["Batch ID", batch['id']],
    ["Batch Type", batch['batch_type']],
    ["Status", batch['status']],
    ["Created", batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(batch_info, headers=["Property", "Value"])

# Get jobs in batch
jobs = get_batch_jobs(mapping_batch_id)
job_count = len(jobs)

ux.info(f"\nTotal jobs: {job_count}")

# Check entity existence for FINISHED/CANCELLED jobs to show what will be resubmitted
from helpers.entity_validator import EntityValidator
from helpers.constants import JobStatus
from helpers.configuration import read_configuration

validator = EntityValidator()

# Get full configuration data for sub-portfolio checking
config_data = None
try:
    config = read_configuration(batch['configuration_id'])
    config_data = config.get('config_data') if config else None
except:
    pass

# Categorize jobs by status and entity existence
pending_jobs = []      # INITIATED jobs - will be submitted
skipped_jobs = []      # FINISHED/CANCELLED jobs where entity still exists - will be skipped
resubmit_jobs = []     # FINISHED/CANCELLED jobs where entity is missing - will be resubmitted
other_jobs = []        # Jobs in other states (SUBMITTED, RUNNING, etc.)

for job in jobs:
    config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
    config_result = execute_query(config_query, (job['job_configuration_id'],))
    job_config_data = config_result.iloc[0]['job_configuration_data'] if not config_result.empty else {}
    portfolio_name = job_config_data.get('Portfolio', 'N/A')
    edm_name = job_config_data.get('Database', 'N/A')
    import_file = job_config_data.get('Import File', 'N/A')
    display_name = f"{portfolio_name} ({edm_name}) - {import_file}"
    
    if job['status'] == JobStatus.INITIATED:
        pending_jobs.append(display_name)
    elif job['status'] in (JobStatus.FINISHED, JobStatus.CANCELLED):
        entity_exists = validator.check_entity_exists_for_job(
            job_config_data, batch['batch_type'], config_data
        )
        if entity_exists:
            skipped_jobs.append(display_name)
        else:
            resubmit_jobs.append(display_name)
    else:
        other_jobs.append((display_name, job['status']))

# Display categorized job summary
ux.subheader("Job Summary")

if pending_jobs:
    ux.info(f"\n🆕 Jobs to execute ({len(pending_jobs)}):")
    for name in pending_jobs[:10]:
        ux.info(f"  • {name}")
    if len(pending_jobs) > 10:
        ux.info(f"  ... and {len(pending_jobs) - 10} more")

if resubmit_jobs:
    ux.warning(f"\n🔄 Jobs to re-execute - sub-portfolios missing ({len(resubmit_jobs)}):")
    for name in resubmit_jobs[:10]:
        ux.warning(f"  • {name}")
    if len(resubmit_jobs) > 10:
        ux.warning(f"  ... and {len(resubmit_jobs) - 10} more")

if skipped_jobs:
    ux.success(f"\n✓ Jobs to skip - sub-portfolios exist ({len(skipped_jobs)}):")
    for name in skipped_jobs[:10]:
        ux.success(f"  • {name}")
    if len(skipped_jobs) > 10:
        ux.success(f"  ... and {len(skipped_jobs) - 10} more")

if other_jobs:
    ux.info(f"\n⏳ Jobs in progress ({len(other_jobs)}):")
    for name, status in other_jobs[:5]:
        ux.info(f"  • {name} ({status})")

# Summary line
total_to_process = len(pending_jobs) + len(resubmit_jobs)
ux.info(f"\n📊 Summary: {total_to_process} job(s) will be executed, {len(skipped_jobs)} will be skipped")

# Validate batch before submission
ux.subheader("Validate Batch")
validation_errors = validate_batch(mapping_batch_id)

if validation_errors:
    validation_failed = True
    ux.error("✗ Batch validation failed:")
    for error in validation_errors:
        ux.error(f"  {error}")
    step.log(f"Batch validation failed: {len(validation_errors)} error(s)")
else:
    ux.success("✓ Batch validation passed")
    step.log(f"Verified batch: {total_to_process} jobs to execute, {len(skipped_jobs)} to skip")

## 4) Execute Portfolio Mapping

In [ ]:
# Execute portfolio mapping SQL scripts
if validation_failed:
    ux.warning("⏭ Skipping execution due to validation failure")
    result = None
    failed_count = 0
else:
    ux.subheader("Execute Portfolio Mapping")

    ux.info("")
    ux.info("Execution Process:")
    ux.info("  - Each job executes a SQL script to create sub-portfolios")
    ux.info("  - SQL scripts are located in workspace/sql/portfolio_mapping/{Cycle Type}/")
    ux.info("  - Script naming: 2b_Query_To_Create_Sub_Portfolios_{Import File}_RMS_BackEnd.sql")
    ux.info("  - This is a synchronous local operation (not submitted to Moody's)")
    ux.info("")

    # Execute
    ux.info("\nExecuting mapping scripts...")

    # Pass step.step_id to associate batch with this step (not the creation step)
    result = submit_batch(mapping_batch_id, IRPClient(), step_id=step.step_id)

    # Display results
    ux.success(f"\n✓ Batch execution completed")
    ux.info(f"  Executed: {result['submitted_jobs']} jobs")
    ux.info(f"  Status: {result['batch_status']}")

    # Check for errors
    failed_count = len([j for j in result['jobs'] if 'error' in j])
    if failed_count > 0:
        ux.warning(f"\n⚠ {failed_count} job(s) failed to execute")
        for job_result in result['jobs']:
            if 'error' in job_result:
                ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

    step.log(f"Batch executed: {result['submitted_jobs']} jobs, {failed_count} failed")

## 5) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

if validation_failed:
    # Handle validation failure
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    from helpers.teams_notification import send_validation_failure_notification
    
    error_message = "\n".join(validation_errors)
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
    
    # Send Teams notification
    send_validation_failure_notification(
        cycle_name=context.cycle_name,
        stage_name=context.stage_name,
        step_name=context.step_name,
        validation_errors=validation_errors,
        notebook_path=str(context.notebook_path)
    )
    
    ux.error("\n" + "="*60)
    ux.error("BATCH VALIDATION FAILED")
    ux.error("="*60)
    ux.info(f"\nBatch ID: {mapping_batch_id}")
    ux.error(f"\nValidation errors ({len(validation_errors)}):")
    for error in validation_errors:
        ux.error(f"  {error}")
    ux.info("\nPlease fix the validation errors and retry.")

elif failed_count > 0:
    # Handle execution failures
    failed_job_errors = [
        f"Job {j['job_id']}: {j['error']}" 
        for j in result['jobs'] if 'error' in j
    ]
    error_message = f"{failed_count} job(s) failed to execute:\n" + "\n".join(failed_job_errors)
    
    from helpers.step import update_step_run
    from helpers.constants import StepStatus
    update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
    
    ux.error("\n" + "="*60)
    ux.error("PORTFOLIO MAPPING BATCH EXECUTION FAILED")
    ux.error("="*60)
    ux.info(f"\nBatch ID: {mapping_batch_id}")
    ux.info(f"Executed: {result['submitted_jobs']} job(s)")
    ux.error(f"Failed: {failed_count} job(s)")
    ux.info("\nFailed jobs:")
    for error in failed_job_errors:
        ux.error(f"  {error}")
    ux.info("\nPlease review the errors and resubmit failed jobs.")

else:
    # Complete the step successfully
    output_data = {
        'batch_id': mapping_batch_id,
        'batch_type': batch['batch_type'],
        'batch_status': result['batch_status'],
        'submitted_jobs': result['submitted_jobs'],
        'failed_jobs': failed_count
    }
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("PORTFOLIO MAPPING BATCH EXECUTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nExecuted {result['submitted_jobs']} job(s)")
    ux.info(f"Batch status: {result['batch_status']}")
    ux.info("\nNext: Proceed to other data loading steps or analysis")